In [318]:
import numpy as np
import doctest
from tqdm import tqdm_notebook as tqdm
from copy import copy
from random import choice
import itertools
from collections import defaultdict

In [282]:
# 0 -> 2
# 1 -> 3
# ...
# 8 -> 10
# 9 -> J
# 10 -> Q
# 11 -> K
# 12 -> A
J, Q, K, A = 'J', 'Q', 'K', 'A'
faces = dict(zip(list(range(2,11))+[J,Q,K,A],range(13)))

In [89]:
cards = list(itertools.product(range(13), range(4)))
combinations = list(itertools.combinations(cards, 5))

In [287]:
prize_dict = {}
for combination in tqdm(combinations):
    prize_dict[combination] = calc_prize(combination)
average_all = sum(prize_dict.values())/len(prize_dict)


In [ ]:
# Ways to win:
# 2 & >=11 -> 1
# 2 & 2 -> 2
# 3 -> 3
# All adjacent -> 4
# Same shape -> 6
# 3 & 2 -> 9
# 4 -> 25
# All adjacent & same shape -> 50
# Same shape & >=10 -> 800

# 4 shapes
# 13 numbers

for combination in combinations:
#     calculate best move + odds for each win for each selection for holding (4!+3!+2!+1!+0!)
    move, odds = best_move(combination)



    

In [334]:
def get_moves(n):
    moves = []
    for i in range(n+2):
        moves.extend(itertools.combinations(range(n), i))
    return moves

In [326]:
def best_move(combination):
#     for each selection for holding
    moves = get_moves(len(combination))
    odds = []
    for move in moves:
        odd = calc_odds(tuple(sorted(combination[m] for m in move)))
        odds.append(odd)
    max_ind = np.argmax(odds)
    return moves[max_ind], odds[max_ind]

In [109]:
def get_other_cards(combination):
    other_cards = copy(cards)
    for card in combination:
        other_cards.remove(card)
    return other_cards


In [298]:
all_odds = copy(prize_dict)
def calc_odds(combination):
    if combination in all_odds:
        return all_odds[combination]
    other_cards = get_other_cards(combination)
    # get all other combinations
    other_combinations = list(itertools.combinations(other_cards, 5-len(combination)))
    # calculate average prize without combination
    prizes = []
    for other_combination in other_combinations:
        other_combination = tuple(sorted(list(other_combination) + list(combination)))
        prizes.append(prize_dict[other_combination])
    odds = sum(prizes)/len(prizes)
    all_odds[combination] = odds
    return odds


In [68]:
def calc_prize(combination):
    nums, shapes = zip(*combination)
    all_same_shape = all_same(shapes)
    if all_same_shape and all(n>=8 for n in nums): return 800
    adjacent = is_adjacent(nums)
    if adjacent and all_same_shape: return 50
    sames, same_nums = are_same(nums)
    if 4 in sames: return 25
    if 3 in sames and 2 in sames: return 9
    if all_same_shape: return 6
    if adjacent: return 4
    if 3 in sames: return 3
    if sames.count(2) == 2: return 2
    if 2 in sames and same_nums[sames.index(2)] >= 9: return 1
    return 0
    

In [373]:
def all_same(lst):
    return lst.count(lst[0]) == len(lst) if lst else False

def is_adjacent(lst):
    '''
    Assuming the combination is sorted, we check the distance between element is 1
    '''
    return all((lst[i+1]-lst[i])%8==1 for i in range(len(lst)-1))

def are_same(lst):
    return zip(*[(lst.count(si),si) for si in set(lst)])

In [382]:
credits = 100
rewards = []
for i in tqdm(range(100000)):
    credits -= 1
    hand = choice(combinations)
    _, reward = best_move(hand)
    rewards.append(reward)
    credits += reward
print(np.mean(rewards))
print(np.std(rewards))

0.9859281856280974
2.7060802099511863


In [301]:
credits

-2275.64604433919

In [386]:
nums = map(lambda x: faces[x], (2,3,5,6,8))
shapes = (0,1,2,0,3)
hand = tuple(zip(nums,shapes))
best_move(hand)

((), 0.3417797888386124)

In [374]:
odds_dict = defaultdict(set)
current_odds = copy(all_odds)
for combination, odd in tqdm(current_odds.items()):
    move,_ = best_move(combination)
    if len(move) != len(combination):
        continue
    nums, shapes = zip(*combination) if combination else ((),())
    odds_dict[odd].add((tuple(nums), all_same(shapes)))

In [381]:

for k,v in sorted(odds_dict.items(), reverse=True):
    print((k, list(v)[0]))
    

(800, ((8, 9, 10, 11, 12), True))
(50, ((0, 1, 10, 11, 12), True))
(25.0, ((12, 12, 12, 12), False))
(19.333333333333332, ((9, 10, 11, 12), True))
(19.270833333333332, ((8, 9, 10, 11), True))
(18.104166666666668, ((8, 9, 10, 12), True))
(9, ((8, 8, 11, 11, 11), False))
(6, ((3, 5, 8, 9, 10), True))
(4.26530612244898, ((8, 8, 8), False))
(4, ((4, 5, 6, 7, 8), False))
(3.6458333333333335, ((0, 10, 11, 12), True))
(3.5833333333333335, ((7, 8, 9, 10), True))
(3.5208333333333335, ((0, 1, 2, 12), True))
(3.4583333333333335, ((2, 3, 4, 5), True))
(2.5833333333333335, ((2, 2, 11, 11), False))
(2.4791666666666665, ((0, 9, 10, 12), True))
(2.4166666666666665, ((0, 1, 10, 12), True))
(2.3541666666666665, ((5, 6, 7, 9), True))
(2.2916666666666665, ((2, 3, 4, 6), True))
(1.5142857142857142, ((12, 12), False))
(1.5127551020408163, ((10, 11, 12), True))
(1.4243197278911566, ((9, 10, 12), True))
(1.4158163265306123, ((8, 9, 10), True))
(1.3273809523809523, ((8, 10, 11), True))
(1.25, ((3, 6, 10, 11), 

In [383]:
len(odds_dict)

55

In [312]:
average_all

0.3417797888386124

In [ ]:
# (800,  Royal Flush
# (50,   Straight Flush
# (25.0, 4
# (19.3, 4 to Royal Flush
# (19.2, 4 to Royal Flush
# (18.1, 4 to Royal Flush
# (9,    Full House
# (6,    Flush
# (4.26, 3
# (4,    Straight
# (3.64, 4 to Straight Flush
# (3.58, 4 to Straight Flush
# (3.52, 4 to Straight Flush
# (3.45, 4 to Straight Flush
# (2.58, 2 pair
# (2.47, 4 to Straight Flush 1 gap
# (2.41, 4 to Straight Flush 1 gap
# (2.35, 4 to Straight Flush 1 gap
# (2.29, 4 to Straight Flush 1 gap
# (1.51, high pair
# (1.51, 3 to Royal Flush
# (1.42, 3 to Royal Flush
# (1.41, 3 to Royal Flush
# (1.32, 3 to Royal Flush
# (1.25, 4 to Flush
# (1.23, 3 to Royal Flush with gaps
# (1.18, 4 to Flush 1 high
# (1.12, 4 to Flush no high
# (0.91, 4 to Straight
# (0.85, 4 to Straight
# (0.79, low pair
# (0.79, 4 to Straight 2 highs
# (0.77, 3 to Straight Flush 
# (0.72, 4 to Straight 1 high
# (0.68, 3 to Straight Flush 1 gap
# (0.68, 3 to Straight Flush 1 high
# (0.66, 4 to Straight no high
# (0.60, 3 to Straight Flush 2 gaps
# (0.59, 3 to Straight Flush 1 high
# (0.58, 3 to Straight Flush no highs
# (0.58, 2 to Royal Flush
# (0.56, 2 to Royal Flush
# (0.55, 2 to Royal Flush
# (0.54, 3 to Straight
# (0.52, 4 to Straight 1 gap
# (0.50, 3 to Flush
# (0.49, 3 to Straight Flush no highs with gaps
# (0.48, 3 to Straight with gaps
# (0.48, 2 to Straight
# (0.47, 2 to Straight with gaps
# (0.45, 2 to Royal Flush 1 high
# (0.45, 2 to Straight with 3 gaps
# (0.44, 1 high
# (0.40, 3 to Flush no high
# (0.34, ((), False))